In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sqlite3
import statsmodels.formula.api as smf

%config InlineBackend.figure_formats = ['svg']
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Load Data

In [43]:
# Specify categorical columns
df = pd.read_csv('../temperament.csv', na_values=' ',
                 dtype={'re_race': 'category', 'lunch': 'category'})
df = df.set_index('id')
df.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
11005,2,0,1,1,61.0,11.0,2.0,19.0,2007.0,2.0,27.0,2007.0,7.824778,0,2,1,0,1,4/24/1999,24,4,1999,9.0,7.0,2/19/2007,1,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.761905,1.909091,1.6,...,NaN,NaN,NaN,1.000000,1.000000,-1,0.116485,-0.2774,-0.4625,-0.053054,-1.11316,0.01796,-0.107848,0.001740,0,-0.033515,-0.054985,0.071977,-0.016677,0.286778,0.120315,-0.445495,0.033515,0.045385,-0.071977,-0.104285,0.016677,-0.009404,-0.009308,0.020195,0.019718,-0.004679,0.0,0.009404,0.012734,-0.020195,-0.029260,0.004679,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11008,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,7.427789,0,2,2,1,0,9/2/1999,2,9,1999,5.0,7.0,2/5/2007,1,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.285714,1.181818,1.4,...,NaN,NaN,NaN,1.000000,1.000000,-1,0.132918,-0.2774,0.5375,0.051265,-0.11316,1.01796,-0.012823,0.115353,0,-0.039809,-0.062327,0.062536,-0.002691,-0.179889,-0.264300,-0.921686,0.039809,0.052727,-0.062536,-0.120718,0.002691,0.007408,0.018535,-0.011637,-0.039528,0.000501,0.0,-0.007408,-0.009812,0.011637,0.022464,-0.000501,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11009,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,8.295688,0,2,2,1,1,10/20/1998,20,10,1998,3.0,8.0,2/5/2007,1,3.0,3.0,3.0,1.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,4.0,3.0,3.0,1.0,1.0,1.0,2.047619,2.090909,2.0,...,NaN,NaN,NaN,1.000000,4.666667,-1,0.004621,-0.2774,-0.3375,0.006330,-0.98816,0.14296,0.014802,-0.002141,0,-0.064393,0.120259,-0.030677,-0.009093,0.631222,-0.033531,-0.159781,0.064393,-0.129859,0.030677,0.007579,0.009093,-0.040247,0.061780,-0.019174,0.002374,-0.005683,0.0,0.040247,-0.081165,0.019174,0.004737,0.005683,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11011,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,7.696099,0,2,2,1,0,5/27/1999,27,5,1999,8.0,7.0,2/5/2007,1,2.0,3.0,4.0,2.0,4.0,3.0,1.0,1.0,4.0,3.0,4.0,5.0,1.0,3.0,2.0,2.0,4.0,2.0,3.0,4.0,1.0,2.761905,2.727273,2.8,...,NaN,NaN,NaN,1.000000,3.000000,-1,0.009721,-0.2774,-0.7125,0.007880,-1.36316,-0.23204,0.013467,0.002292,0,-0.045452,0.034538,-0.049376,0.078814,0.620111,0.043392,0.554505,0.045452,-0.044138,0.049376,0.002479,-0.078814,-0.027903,0.01

In [34]:
#To standardize all columns in a dataframe
for column in df.columns:
  if column.startswith('mt') or column.startswith('c_'):
    df[f'std_{column}'] = (df[column] - df[column].mean()) / df[column].std()

# Select rows for Cohort 1 with no missing data

## For analyses on aggression

In [44]:
aggdf = df[(df.cohort == 1) & (~df.mt1tovertagg.isnull()) & (~df.mt2tovertagg.isnull()) &
           (~df.mt1overtvic.isnull()) & (~df.mt1tovertvic.isnull()) & (~df.mt1picontrol.isnull())]
aggdf.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
1002,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.039014,0,2,2,1,0,12/28/1998,28,12,1998,0.0,7.0,1/11/2006,1,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,3.0,5.0,5.0,5.0,5.0,3.571429,4.818182,2.2,...,-1.135892,1.852989,0.635096,1.000000,1.000000,-1,-0.088186,-0.2774,-0.0875,0.018604,-0.73816,0.39296,0.079563,-0.042355,1,-0.041757,-0.082457,-0.011814,-0.040029,-0.921556,-0.264300,1.364029,0.041757,0.072857,0.011814,0.100386,0.040029,0.038740,0.085678,0.010961,0.091630,0.037137,0.0,-0.038740,-0.067594,-0.010961,-0.093133,-0.037137,2.498208,2.498208,-0.220192,0.0,0.0,0.0,-2.302237,0.0,2.909522,-0.393039,2.883833,-0.385625
1004,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.468857,0,2,6,0,0,7/24/1998,24,7,1998,5.0,7.0,1/11/2006,1,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,1.476190,1.909091,1.0,...,-1.619677,0.999122,0.910435,1.000000,2.666667,-1,0.089335,-0.2774,0.0375,-0.003319,-0.61316,0.51796,-0.042759,0.036120,1,-0.016547,-0.086659,-0.012865,0.009341,-0.274333,-0.264300,-0.731210,0.016547,0.077059,0.012865,-0.077135,-0.009341,0.004642,0.033956,0.003609,-0.035005,-0.002621,0.0,-0.004642,-0.021618,-0.003609,0.021639,0.002621,-0.762949,-0.762949,-0.029049,0.0,0.0,0.0,0.209302,0.0,-0.381589,-0.393039,-0.377324,-0.385625
1006,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.690623,0,2,2,1,1,5/4/1998,4,5,1998,8.0,7.0,1/11/2006,1,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,4.523810,4.727273,4.3,...,2.452864,-0.439749,-0.332146,1.000000,1.000000,-1,0.021370,-0.2774,-0.4625,-0.000969,-1.11316,0.01796,-0.001971,0.000032,1,-0.067819,0.067157,-0.060559,0.041535,0.622889,0.120315,2.316410,0.067819,-0.076757,0.060559,-0.009170,-0.041535,-0.041823,0.033940,-0.037346,0.010800,0.025614,0.0,0.041823,-0.047335,0.037346,-0.005655,-0.025614,1.912953,1.912953,-0.893033,0.0,0.0,0.0,1.191557,0.0,2.806675,-0.875793,2.781922,-0.868969
1007,1,0,1,1,17.0,1.0,1.0,9.0,2006.0,1.0,11.0,2006.0,7.403149,0,2,3,0,0,8/15/1998,15,8,1998,4.0,7.0,1/9/2006,1,3.0,1.0,5.0,1.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,2.0,1.952381,2.090909,1.8,...,0.739859,0.489876,-0.021501,1.666667,2.000000,-1,-

# Select necessary columns for the analyses

## For analyses on aggression

In [45]:
oaggdf = aggdf[['t1age', 're_race', 'lunch', 'sex', 'mt1tovertagg', 'mt2tovertagg',
                'mt1overtvic', 'mt1tovertvic', 'mt1picontrol']]
oaggdf = oaggdf.copy()

In [46]:
oaggdf.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol
id,,,,,,,,,
6085,8.208077,1,1,0,3.25,3.75,2.818182,2.111111,3.375
1002,7.039014,1,0,0,1.00,1.00,4.818182,1.363636,3.250
1004,7.468857,0,0,0,1.00,2.50,1.909091,1.363636,3.375
1006,7.690623,1,1,0,1.00,1.00,4.727273,1.090909,2.875
1007,7.403149,0,0,0,1.50,2.00,2.090909,1.636364,3.125


In [47]:
oaggdf.dtypes

t1age            float64
re_race         category
lunch           category
sex                int64
mt1tovertagg     float64
mt2tovertagg     float64
mt1overtvic      float64
mt1tovertvic     float64
mt1picontrol     float64
dtype: object

In [48]:
oaggdf.shape

(283, 9)

# Creating composite scores

In [49]:
oaggdf['std_mt1overtvic'] = (oaggdf['mt1overtvic'] - oaggdf['mt1overtvic'].mean()) / oaggdf['mt1overtvic'].std()

In [50]:
oaggdf.std_mt1overtvic.describe()

count    2.830000e+02
mean    -3.138440e-17
std      1.000000e+00
min     -1.410061e+00
25%     -7.929774e-01
50%     -1.758942e-01
75%      4.411891e-01
max      2.909522e+00
Name: std_mt1overtvic, dtype: float64

In [51]:
oaggdf['std_mt1tovertvic'] = (oaggdf['mt1tovertvic'] - oaggdf['mt1tovertvic'].mean()) / oaggdf['mt1tovertvic'].std()

In [52]:
oaggdf.std_mt1tovertvic.describe()

count    2.830000e+02
mean     3.263977e-16
std      1.000000e+00
min     -1.036711e+00
25%     -7.148750e-01
50%     -2.321211e-01
75%      4.115506e-01
max      4.595417e+00
Name: std_mt1tovertvic, dtype: float64

In [53]:
oaggdf['comp_overt'] = oaggdf['std_mt1overtvic'] + oaggdf['std_mt1tovertvic']

In [54]:
oaggdf['c_overt'] = oaggdf['comp_overt'] - oaggdf['comp_overt'].mean()
oaggdf['c_icontrol'] = oaggdf['mt1picontrol'] - oaggdf['mt1picontrol'].mean()

In [55]:
oaggdf.describe()

,t1age,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_icontrol
count,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,2.830000e+02,2.830000e+02,2.830000e+02,283.000000,2.830000e+02
mean,7.945891,0.558304,1.448763,1.552120,2.246386,1.585680,3.337498,-3.138440e-17,3.263977e-16,3.012902e-16,0.000000,-3.138440e-17
std,0.319879,0.497469,0.893747,0.946098,0.883924,0.564941,0.585106,1.000000e+00,1.000000e+00,1.551871e+00,1.551871,5.851056e-01
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.375000,-1.410061e+00,-1.036711e+00,-2.446772e+00,-2.446772,-1.962498e+00
25%,7.691992,0.000000,1.000000,1.000000,1.545455,1.181818,3.000000,-7.929774e-01,-7.148750e-01,-1.127946e+00,-1.127946,-3.374979e-01
50%,7.967146,1.000000,1.000000,1.000000,2.090909,1.454545,3.375000,-1.758942e-01,-2.321211e-01,-4.080153e-01,-0.408015,3.750210e-02
75%,8.171116,1.000000,1.500000,1.750000,2.636364,1.818182,3.750000,4.411891e-01,4.115506e-01,8.188107e-01,0.818811,4.125021e-01
max,9.117043,1.000000,5.000000,5.000000,4.818182,4.181818,4.875000,2.909522e+00,4.595417e+00,5.036606e+00,5.036606,1.537502e+00


# Creating interaction terms

In [56]:
oaggdf['overtxicontrol'] = oaggdf['c_overt'] * oaggdf['c_icontrol']

oaggdf['overtxsex'] = oaggdf['c_overt'] * oaggdf['sex']

oaggdf['icontrolsex'] = oaggdf['c_icontrol'] * oaggdf['sex']

oaggdf['overtxicontrolxsex'] = oaggdf['c_overt'] * oaggdf['c_icontrol'] * oaggdf['sex']


# Regression

## Overt victimization predicting overt aggression

In [59]:
oaggovic = smf.ols(formula=
                   f'mt2tovertagg ~ mt1tovertagg + c_overt + c_icontrol + sex + overtxicontrol + overtxsex + icontrolsex + overtxicontrolxsex', data=oaggdf).fit()
oaggovic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           mt2tovertagg   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.392
Method:                 Least Squares   F-statistic:                     23.69
Date:                Mon, 08 Apr 2019   Prob (F-statistic):           1.66e-27
Time:                        17:38:43   Log-Likelihood:                -310.99
No. Observations:                 283   AIC:                             640.0
Df Residuals:                     274   BIC:                             672.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.9095      0.117      7.770      0.000       0.679       1.140
mt1tovertagg           0.5282      0.060      8.803      0.000       0.410       0.646
c_overt               -0.0214      0.056     -0.381      0.703      -0.132       0.089
c_icontrol            -0.2670      0.115     -2.313      0.021      -0.494      -0.040
sex                   -0.2179      0.093     -2.334      0.020      -0.402      -0.034
overtxicontrol         0.0907      0.086      1.055      0.293      -0.079       0.260
overtxsex              0.1252      0.066      1.904      0.058      -0.004       0.255
icontrolsex           -0.0346      0.158     -0.218      0.827      -0.346       0.277
overtxicontrolxsex    -0.2432      0.104     -2.334      0.020      -0.448      -0.038
==============================================================================
Omnibus:                       58.083   Durbin-Watson:                   2.050
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.946
Skew:                           1.101   Prob(JB):                     1.63e-23
Kurtosis:                       5.012   Cond. No.                         8.82
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Decomposing the 3-way interaction by sex

In [61]:
df = oaggdf[(oaggdf.sex == 1)].copy()
df.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_icontrol,overtxicontrol,overtxsex,icontrolsex,overtxicontrolxsex
id,,,,,,,,,,,,,,,,,,
1003,7.660507,1,0,1,1.0,1.00,1.363636,1.000000,3.125,-0.998672,-1.036711,-2.035383,-2.035383,-0.212498,0.432515,-2.035383,-0.212498,0.432515
1008,7.403149,1,1,1,1.0,1.00,1.636364,1.636364,2.000,-0.690130,0.089715,-0.600415,-0.600415,-1.337498,0.803054,-0.600415,-1.337498,0.803054
1011,7.698836,0,1,1,1.0,2.75,3.363636,1.181818,3.000,1.263967,-0.714875,0.549092,0.549092,-0.337498,-0.185317,0.549092,-0.337498,-0.185317
1012,7.603012,1,1,1,1.0,1.00,1.818182,1.090909,4.000,-0.484436,-0.875793,-1.360229,-1.360229,0.662502,-0.901154,-1.360229,0.662502,-0.901154
1014,7.701574,1,1,1,1.0,1.00,3.181818,1.545455,3.000,1.058272,-0.071203,0.987069,0.987069,-0.337498,-0.333134,0.987069,-0.337498,-0.333134


In [62]:
df.shape

(158, 18)

Exploring...
I'm trying to see if standardizing all columns up front would work. Would I get the same output if the dependent variable and covariate are also standardized?

In [35]:
for column in girls.columns:
  if column.startswith('mt'):
    girls[f'std_{column}'] = (girls[column] - girls[column].mean()) / girls[column].std()

In [27]:
#girls['std_mt1overtvic'] = (girls['mt1overtvic'] - girls['mt1overtvic'].mean()) / girls['mt1overtvic'].std()

In [28]:
#girls['std_mt1tovertvic'] = (girls['mt1tovertvic'] - girls['mt1tovertvic'].mean()) / girls['mt1tovertvic'].std()

In [36]:
girls.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_negemot,overtxnegemot,overtxsex,negemotsex,overtxnegemotxsex,std_mt1psmfq,std_mt2psmfq,std_mt1pnegemot
count,136.000000,136.0,136.000000,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02
mean,7.934714,1.0,1.195324,1.198388,2.253342,1.472594,2.533254,-4.832736e-16,-6.530724e-18,-5.028657e-16,6.530724e-18,-8.979745e-17,0.185853,-0.183935,-0.082753,0.189440,-3.265362e-16,-3.134747e-16,-1.469413e-16
std,0.281883,0.0,0.268510,0.252010,0.935011,0.501582,0.600387,1.000000e+00,1.000000e+00,1.629323e+00,1.629323e+00,6.003874e-01,1.348208,1.580783,0.600387,1.233063,1.000000e+00,1.000000e+00,1.000000e+00
min,7.389459,1.0,1.000000,1.000000,1.000000,1.000000,1.100000,-1.340457e+00,-9.422052e-01,-2.282662e+00,-2.282662e+00,-1.433254e+00,-5.051066,-2.426519,-1.516007,-4.950146,-7.274388e-01,-7.872217e-01,-2.387215e+00
25%,7.700890,1.0,1.000000,1.000000,1.545455,1.090909,2.152083,-7.570898e-01,-7.609607e-01,-1.162167e+00,-1.162167e+00,-3.811702e-01,-0.264937,-1.335832,-0.463924,-0.235100,-7.274388e-01,-7.872217e-01,-6.348738e-01
50%,7.973990,1.0,1.076923,1.115385,2.000000,1.363636,2.512500,-2.709509e-01,-2.172270e-01,-4.173721e-01,-4.173721e-01,-2.075358e-02,-0.023954,-0.588728,-0.103507,0.007464,-4.409571e-01,-3.293650e-01,-3.456698e-02
75%,8.145791,1.0,1.250000,1.307692,2.750000,1.636364,2.839583,5.311782e-01,3.265068e-01,7.406407e-01,7.406407e-01,3.063298e-01,0.324524,0.600480,0.223576,0.411626,2.036267e-01,4.337294e-01,5.102201e-01
max,8.618754,1.0,2.538462,2.750000,4.818182,3.545455,4.344444,2.743110e+00,4.132643e+00,5.222881e+00,5.222881e+00,1.811191e+00,7.457490,4.632471,1.728438,6.602834,5.002195e+00,6.156938e+00,3.016704e+00


In [37]:
girls['comp_overt'] = girls['std_mt1overtvic'] + girls['std_mt1tovertvic']

In [38]:
girls['c_overt'] = girls['comp_overt'] - girls['comp_overt'].mean()
girls['c_negemot'] = girls['std_mt1pnegemot'] - girls['std_mt1pnegemot'].mean()

In [39]:
girls['overtxnegemot'] = girls['c_overt'] * girls['c_negemot']

In [40]:
girls.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_negemot,overtxnegemot,overtxsex,negemotsex,overtxnegemotxsex,std_mt1psmfq,std_mt2psmfq,std_mt1pnegemot
count,136.000000,136.0,136.000000,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02
mean,7.934714,1.0,1.195324,1.198388,2.253342,1.472594,2.533254,-4.832736e-16,-6.530724e-18,-5.028657e-16,6.530724e-18,-6.530724e-18,0.309555,-0.183935,-0.082753,0.189440,-3.265362e-16,-3.134747e-16,-1.469413e-16
std,0.281883,0.0,0.268510,0.252010,0.935011,0.501582,0.600387,1.000000e+00,1.000000e+00,1.629323e+00,1.629323e+00,1.000000e+00,2.245564,1.580783,0.600387,1.233063,1.000000e+00,1.000000e+00,1.000000e+00
min,7.389459,1.0,1.000000,1.000000,1.000000,1.000000,1.100000,-1.340457e+00,-9.422052e-01,-2.282662e+00,-2.282662e+00,-2.387215e+00,-8.413011,-2.426519,-1.516007,-4.950146,-7.274388e-01,-7.872217e-01,-2.387215e+00
25%,7.700890,1.0,1.000000,1.000000,1.545455,1.090909,2.152083,-7.570898e-01,-7.609607e-01,-1.162167e+00,-1.162167e+00,-6.348738e-01,-0.441276,-1.335832,-0.463924,-0.235100,-7.274388e-01,-7.872217e-01,-6.348738e-01
50%,7.973990,1.0,1.076923,1.115385,2.000000,1.363636,2.512500,-2.709509e-01,-2.172270e-01,-4.173721e-01,-4.173721e-01,-3.456698e-02,-0.039898,-0.588728,-0.103507,0.007464,-4.409571e-01,-3.293650e-01,-3.456698e-02
75%,8.145791,1.0,1.250000,1.307692,2.750000,1.636364,2.839583,5.311782e-01,3.265068e-01,7.406407e-01,7.406407e-01,5.102201e-01,0.540524,0.600480,0.223576,0.411626,2.036267e-01,4.337294e-01,5.102201e-01
max,8.618754,1.0,2.538462,2.750000,4.818182,3.545455,4.344444,2.743110e+00,4.132643e+00,5.222881e+00,5.222881e+00,3.016704e+00,12.421131,4.632471,1.728438,6.602834,5.002195e+00,6.156938e+00,3.016704e+00


In [41]:
g_depovic = smf.ols(formula=f'std_mt2psmfq ~ std_mt1psmfq + c_overt + c_negemot + overtxnegemot', data=girls).fit()
g_depovic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           std_mt2psmfq   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     19.64
Date:                Fri, 05 Apr 2019   Prob (F-statistic):           1.11e-12
Time:                        16:30:11   Log-Likelihood:                -160.53
No. Observations:                 136   AIC:                             331.1
Df Residuals:                     131   BIC:                             345.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.0318      0.070     -0.457      0.649      -0.170       0.106
std_mt1psmfq      0.1363      0.092      1.479      0.141      -0.046       0.319
c_overt           0.0787      0.046      1.728      0.086      -0.011       0.169
c_negemot         0.3291      0.083      3.965      0.000       0.165       0.493
overtxnegemot     0.1029      0.036      2.839      0.005       0.031       0.175
==============================================================================
Omnibus:                       66.589   Durbin-Watson:                   2.330
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.007
Skew:                           1.790   Prob(JB):                     1.28e-57
Kurtosis:                       8.782   Cond. No.                         3.81
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Creating High vs. Low IC groups

In [251]:
boys['icabove'] = boys['c_icontrol'] - boys['c_icontrol'].std()
boys['icbelow'] = boys['c_icontrol'] + boys['c_icontrol'].std()

In [9]:
oaggdf.describe()

,t1age,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol
count,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
mean,7.945891,0.558304,1.448763,1.552120,2.246386,1.585680,3.337498
std,0.319879,0.497469,0.893747,0.946098,0.883924,0.564941,0.585106
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.375000
25%,7.691992,0.000000,1.000000,1.000000,1.545455,1.181818,3.000000
50%,7.967146,1.000000,1.000000,1.000000,2.090909,1.454545,3.375000
75%,8.171116,1.000000,1.500000,1.750000,2.636364,1.818182,3.750000
max,9.117043,1.000000,5.000000,5.000000,4.818182,4.181818,4.875000


## For analyses on depression

In [10]:
depdf = df[(df.cohort == 1) & (~df.mt1psmfq.isnull()) & (~df.mt2psmfq.isnull()) &
           (~df.mt1overtvic.isnull()) & (~df.mt1tovertvic.isnull()) & (~df.mt1pnegemot.isnull())]
depdf.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,std_mt5urgency,std_mt5senseek,std_mt5dpga,std_mt5trust,std_mt5communication,std_mt5alienation,std_mt5tvictim,std_mt5tovertvic,std_mt5trelationvic,std_mt5tverbalvic,std_mt5tphysicalvic,std_mt5trelvicfriend,std_mt5trelvicotherkid,std_mt5trelationagg,std_mt5tovertagg,std_mt5tprosocial,std_mt5ticontrol,std_mt5tsoothe,std_mt5tsoother,std_mt5tsadness,std_mt5tanger,std_mt5tattfocus,std_mt5timpulsivity,std_mt5tnegemot,std_mt5tantisocial,std_mt5tsochelp,std_mt5texclusion,std_mt5tanxsolitude,std_mt5tbrief,std_mt5tbriefinhibit,std_mt5tbriefplan,std_mt5tbriefattention,std_mt5tbriefmemory,std_mt5academic,std_mt5tproactiveagg,std_mt5treactiveagg,std_mt5tdpga,std_c_solve,std_c_t1tovertagg,std_c_icontrol,std_c_icontrolabove,std_c_icontrolbelow,std_c_humor,std_c_revenge,std_c_ignore,std_c_adult,std_c_negemot,std_c_t1psmfq,std_c_t1vic,std_c_overt
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,0.620996,-1.176079,0.296541,-1.325480,-0.548046,0.013611,-0.202282,0.323542,-0.742643,0.554748,0.001668,-0.873994,-0.536959,-0.245134,2.034101,-1.825258,-1.081519,-0.601140,0.601140,-0.154772,0.377853,-1.098385,0.863720,0.266892,1.565047,1.723827,0.686411,-0.317519,1.981622,1.234132,1.844161,2.760157,1.569477,-1.131951,0.565928,1.023935,0.632415,-0.957633,1.926851,0.091851,0.091851,0.091851,NaN,0.582090,-1.570972,0.070529,-0.139544,-0.518611,0.196174,1.012607
6019,1,0,1,1,9.0,6.0,2.0,9.0,2006.0,2.0,23.0,2006.0,8.599589,0,2,2,1,0,7/5/1997,5,7,1997,7.0,8.0,2/9/2006,1,2.0,3.0,2.0,2.0,3.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.952381,1.727273,2.2,...,0.264917,0.991838,-0.226746,0.499063,0.407316,-0.304567,-0.694011,-0.753857,-0.580210,-0.768060,-0.635488,-0.873994,-0.213938,-0.967769,-0.479531,-1.484638,0.380883,NaN,NaN,0.408445,NaN,0.085206,-0.237825,NaN,-0.581851,0.700025,1.459318,0.611923,-0.599051,-0.650868,-0.343233,-0.307732,-0.819923,0.952754,-0.480126,-0.679650,-0.617689,0.657861,-0.528002,0.302247,0.302247,0.302247,0.926993,-0.839157,1.032801,-2.078722,-0.033359,-0.249964,-0.231497,-0.206451
1002,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.039014,0,2,2,1,0,12/28/1998,28,12,1998,0.0,7.0,1/11/2006,1,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,3.0,5.0,5.0,5.0,5.0,3.571429,4.818182,2.2,...,-1.159400,-0.675791,-0.750032,1.060460,1.499159,-1.259102,-0.857920,-0.753857,-0.905076,-0.768060,-0.635488,-0.873994,-0.859980,-0.726891,-0.479531,0.899700,0.543372,1.025194,-1.025194,-0.949902,-0.392959,1.032079,-0.153091,-0.856461,-0.455563,-0.730674,-0.472951,-0.317519,-0.115175,-0.650868,0.686129,-0.307732,-0.620806,0.170990,-0.480126,-0.338933,-0.617689,-1.528086,-0.528002,-0.118545,-0.118545,-0.118545,-0.710520,-0.971856,-0.335266,-1.321168,-1.568216,-0.787257,1.845759,1.600183
1004,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.468857,0,2,6,0,0,7/24/1998,24,7,1998,5.0,7.0,1/11/2006,1,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,1.476190,1.909091,1.0,...,0.442956,0.324786,-0.226746,1.060460,1.499159,-0.198507,-0.366191,-0.138200,-0.580210,0.025625,-0.316910,-0.873994,-0.213938,-0.245134,0.148877,0.218460,-0.594052,0.212027,-0.212027,-0.419816,-0.547121,-0.388230,-0.153091,-0.428517,-0.455563,-0.730674,-0.472951,-0.549879,-0.437759,-0.462368,-0.343233,-0.648609,-0.222573,0.170

## For analyses on depression

In [11]:
depdf = depdf[['t1age', 're_race', 'lunch', 'sex', 'mt1psmfq', 'mt2psmfq',
                'mt1overtvic', 'mt1tovertvic', 'mt1pnegemot']]
depdf.head()

,t1age,re_race,lunch,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot
id,,,,,,,,,
6085,8.208077,1,1,0,1.076923,1.153846,2.818182,2.111111,2.516667
6019,8.599589,1,0,0,1.153846,1.230769,1.727273,1.727273,2.577778
1002,7.039014,1,0,0,1.000000,1.000000,4.818182,1.363636,1.694444
1004,7.468857,0,0,0,1.000000,1.000000,1.909091,1.363636,2.341667
1006,7.690623,1,1,0,1.384615,1.538462,4.727273,1.090909,3.238889


In [12]:
depdf.dtypes

t1age            float64
re_race         category
lunch           category
sex                int64
mt1psmfq         float64
mt2psmfq         float64
mt1overtvic      float64
mt1tovertvic     float64
mt1pnegemot      float64
dtype: object

In [13]:
depdf.shape

(242, 9)

In [14]:
depdf.mt2psmfq.count()

242

In [15]:
depdf.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,7.937912,0.561983,1.224650,1.224968,2.245680,1.581225,2.616007
std,0.326755,0.497171,0.292352,0.263350,0.892043,0.564251,0.606103
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.100000
25%,7.682409,0.000000,1.000000,1.000000,1.545455,1.090909,2.187500
50%,7.964408,1.000000,1.153846,1.153846,2.090909,1.454545,2.545833
75%,8.168378,1.000000,1.307692,1.307692,2.636364,1.818182,3.013194
max,9.117043,1.000000,2.923077,2.750000,4.818182,4.181818,4.344444


# Creating composite scores

In [16]:
depdf['std_mt1overtvic'] = (depdf['mt1overtvic'] - depdf['mt1overtvic'].mean()) / depdf['mt1overtvic'].std()

In [17]:
depdf.std_mt1overtvic.describe()

count    2.420000e+02
mean     1.504765e-16
std      1.000000e+00
min     -1.396435e+00
25%     -7.849685e-01
50%     -1.735016e-01
75%      4.379652e-01
max      2.883833e+00
Name: std_mt1overtvic, dtype: float64

In [18]:
depdf['std_mt1tovertvic'] = (depdf['mt1tovertvic'] - depdf['mt1tovertvic'].mean()) / depdf['mt1tovertvic'].std()

In [19]:
depdf.std_mt1tovertvic.describe()

count    2.420000e+02
mean    -5.872254e-17
std      1.000000e+00
min     -1.030084e+00
25%     -8.689689e-01
50%     -2.245101e-01
75%      4.199486e-01
max      4.608931e+00
Name: std_mt1tovertvic, dtype: float64

In [20]:
depdf['comp_overt'] = depdf['std_mt1overtvic'] + depdf['std_mt1tovertvic']

In [21]:
depdf['c_overt'] = depdf['comp_overt'] - depdf['comp_overt'].mean()
depdf['c_negemot'] = depdf['mt1pnegemot'] - depdf['mt1pnegemot'].mean()

In [22]:
depdf.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_negemot
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,2.420000e+02,2.420000e+02,2.420000e+02,2.420000e+02,2.420000e+02
mean,7.937912,0.561983,1.224650,1.224968,2.245680,1.581225,2.616007,1.504765e-16,-5.872254e-17,8.808381e-17,2.936127e-17,-1.981886e-16
std,0.326755,0.497171,0.292352,0.263350,0.892043,0.564251,0.606103,1.000000e+00,1.000000e+00,1.561202e+00,1.561202e+00,6.061028e-01
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.100000,-1.396435e+00,-1.030084e+00,-2.426519e+00,-2.426519e+00,-1.516007e+00
25%,7.682409,0.000000,1.000000,1.000000,1.545455,1.090909,2.187500,-7.849685e-01,-8.689689e-01,-1.101674e+00,-1.101674e+00,-4.285069e-01
50%,7.964408,1.000000,1.153846,1.153846,2.090909,1.454545,2.545833,-1.735016e-01,-2.245101e-01,-3.815158e-01,-3.815158e-01,-7.017360e-02
75%,8.168378,1.000000,1.307692,1.307692,2.636364,1.818182,3.013194,4.379652e-01,4.199486e-01,8.120435e-01,8.120435e-01,3.971875e-01
max,9.117043,1.000000,2.923077,2.750000,4.818182,4.181818,4.344444,2.883833e+00,4.608931e+00,5.046896e+00,5.046896e+00,1.728438e+00


# Creating interaction terms

In [23]:
depdf['overtxnegemot'] = depdf['c_overt'] * depdf['c_negemot']

depdf['overtxsex'] = depdf['c_overt'] * depdf['sex']

depdf['negemotsex'] = depdf['c_negemot'] * depdf['sex']

depdf['overtxnegemotxsex'] = depdf['c_overt'] * depdf['c_negemot'] * depdf['sex']


# Regression

## Overt victimization predicting overt depression

In [24]:
depovic = smf.ols(formula=f'mt2psmfq ~ mt1psmfq + c_overt + c_negemot + sex + overtxnegemot + \
                           overtxsex + negemotsex + overtxnegemotxsex', data=depdf).fit()
depovic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               mt2psmfq   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     13.54
Date:                Fri, 05 Apr 2019   Prob (F-statistic):           4.50e-16
Time:                        16:16:35   Log-Likelihood:                 26.213
No. Observations:                 242   AIC:                            -34.43
Df Residuals:                     233   BIC:                            -3.026
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             1.0535      0.081     13.022      0.000       0.894       1.213
mt1psmfq              0.1473      0.063      2.330      0.021       0.023       0.272
c_overt               0.0277      0.015      1.838      0.067      -0.002       0.057
c_negemot             0.1495      0.041      3.611      0.000       0.068       0.231
sex                  -0.0234      0.030     -0.791      0.430      -0.082       0.035
overtxnegemot        -0.0561      0.025     -2.217      0.028      -0.106      -0.006
overtxsex            -0.0043      0.020     -0.219      0.827      -0.043       0.034
negemotsex           -0.0063      0.050     -0.127      0.899      -0.104       0.091
overtxnegemotxsex     0.1007      0.030      3.316      0.001       0.041       0.161
==============================================================================
Omnibus:                       60.263   Durbin-Watson:                   2.211
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.098
Skew:                           1.168   Prob(JB):                     1.03e-30
Kurtosis:                       5.870   Cond. No.                         13.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Decomposing the 3-way interaction by sex

In [25]:
girls = depdf[(depdf.sex == 1)].copy()
girls.head()

,t1age,re_race,lunch,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_negemot,overtxnegemot,overtxsex,negemotsex,overtxnegemotxsex
id,,,,,,,,,,,,,,,,,,
2053,8.106776,1,0,1,1.153846,1.000000,1.454545,1.000000,2.194444,-0.886880,-1.030084,-1.916963,-1.916963,-0.421562,0.808120,-1.916963,-0.421562,0.808120
1003,7.660507,1,0,1,1.230769,1.307692,1.363636,1.000000,3.032407,-0.988791,-1.030084,-2.018874,-2.018874,0.416400,-0.840660,-2.018874,0.416400,-0.840660
1008,7.403149,1,1,1,1.230769,1.000000,1.636364,1.636364,3.266667,-0.683057,0.097719,-0.585338,-0.585338,0.650660,-0.380856,-0.585338,0.650660,-0.380856
1011,7.698836,0,1,1,1.000000,1.000000,3.363636,1.181818,1.466667,1.253254,-0.707854,0.545400,0.545400,-1.149340,-0.626850,0.545400,-1.149340,-0.626850
1012,7.603012,1,1,1,1.000000,1.000000,1.818182,1.090909,2.702778,-0.479235,-0.868969,-1.348204,-1.348204,0.086771,-0.116985,-1.348204,0.086771,-0.116985


In [26]:
girls.shape

(136, 18)

Exploring...
I'm trying to see if standardizing all columns up front would work. Would I get the same output if the dependent variable and covariate are also standardized?

In [35]:
for column in girls.columns:
  if column.startswith('mt'):
    girls[f'std_{column}'] = (girls[column] - girls[column].mean()) / girls[column].std()

In [27]:
#girls['std_mt1overtvic'] = (girls['mt1overtvic'] - girls['mt1overtvic'].mean()) / girls['mt1overtvic'].std()

In [28]:
#girls['std_mt1tovertvic'] = (girls['mt1tovertvic'] - girls['mt1tovertvic'].mean()) / girls['mt1tovertvic'].std()

In [36]:
girls.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_negemot,overtxnegemot,overtxsex,negemotsex,overtxnegemotxsex,std_mt1psmfq,std_mt2psmfq,std_mt1pnegemot
count,136.000000,136.0,136.000000,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02
mean,7.934714,1.0,1.195324,1.198388,2.253342,1.472594,2.533254,-4.832736e-16,-6.530724e-18,-5.028657e-16,6.530724e-18,-8.979745e-17,0.185853,-0.183935,-0.082753,0.189440,-3.265362e-16,-3.134747e-16,-1.469413e-16
std,0.281883,0.0,0.268510,0.252010,0.935011,0.501582,0.600387,1.000000e+00,1.000000e+00,1.629323e+00,1.629323e+00,6.003874e-01,1.348208,1.580783,0.600387,1.233063,1.000000e+00,1.000000e+00,1.000000e+00
min,7.389459,1.0,1.000000,1.000000,1.000000,1.000000,1.100000,-1.340457e+00,-9.422052e-01,-2.282662e+00,-2.282662e+00,-1.433254e+00,-5.051066,-2.426519,-1.516007,-4.950146,-7.274388e-01,-7.872217e-01,-2.387215e+00
25%,7.700890,1.0,1.000000,1.000000,1.545455,1.090909,2.152083,-7.570898e-01,-7.609607e-01,-1.162167e+00,-1.162167e+00,-3.811702e-01,-0.264937,-1.335832,-0.463924,-0.235100,-7.274388e-01,-7.872217e-01,-6.348738e-01
50%,7.973990,1.0,1.076923,1.115385,2.000000,1.363636,2.512500,-2.709509e-01,-2.172270e-01,-4.173721e-01,-4.173721e-01,-2.075358e-02,-0.023954,-0.588728,-0.103507,0.007464,-4.409571e-01,-3.293650e-01,-3.456698e-02
75%,8.145791,1.0,1.250000,1.307692,2.750000,1.636364,2.839583,5.311782e-01,3.265068e-01,7.406407e-01,7.406407e-01,3.063298e-01,0.324524,0.600480,0.223576,0.411626,2.036267e-01,4.337294e-01,5.102201e-01
max,8.618754,1.0,2.538462,2.750000,4.818182,3.545455,4.344444,2.743110e+00,4.132643e+00,5.222881e+00,5.222881e+00,1.811191e+00,7.457490,4.632471,1.728438,6.602834,5.002195e+00,6.156938e+00,3.016704e+00


In [37]:
girls['comp_overt'] = girls['std_mt1overtvic'] + girls['std_mt1tovertvic']

In [38]:
girls['c_overt'] = girls['comp_overt'] - girls['comp_overt'].mean()
girls['c_negemot'] = girls['std_mt1pnegemot'] - girls['std_mt1pnegemot'].mean()

In [39]:
girls['overtxnegemot'] = girls['c_overt'] * girls['c_negemot']

In [40]:
girls.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_negemot,overtxnegemot,overtxsex,negemotsex,overtxnegemotxsex,std_mt1psmfq,std_mt2psmfq,std_mt1pnegemot
count,136.000000,136.0,136.000000,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,1.360000e+02,136.000000,136.000000,136.000000,136.000000,1.360000e+02,1.360000e+02,1.360000e+02
mean,7.934714,1.0,1.195324,1.198388,2.253342,1.472594,2.533254,-4.832736e-16,-6.530724e-18,-5.028657e-16,6.530724e-18,-6.530724e-18,0.309555,-0.183935,-0.082753,0.189440,-3.265362e-16,-3.134747e-16,-1.469413e-16
std,0.281883,0.0,0.268510,0.252010,0.935011,0.501582,0.600387,1.000000e+00,1.000000e+00,1.629323e+00,1.629323e+00,1.000000e+00,2.245564,1.580783,0.600387,1.233063,1.000000e+00,1.000000e+00,1.000000e+00
min,7.389459,1.0,1.000000,1.000000,1.000000,1.000000,1.100000,-1.340457e+00,-9.422052e-01,-2.282662e+00,-2.282662e+00,-2.387215e+00,-8.413011,-2.426519,-1.516007,-4.950146,-7.274388e-01,-7.872217e-01,-2.387215e+00
25%,7.700890,1.0,1.000000,1.000000,1.545455,1.090909,2.152083,-7.570898e-01,-7.609607e-01,-1.162167e+00,-1.162167e+00,-6.348738e-01,-0.441276,-1.335832,-0.463924,-0.235100,-7.274388e-01,-7.872217e-01,-6.348738e-01
50%,7.973990,1.0,1.076923,1.115385,2.000000,1.363636,2.512500,-2.709509e-01,-2.172270e-01,-4.173721e-01,-4.173721e-01,-3.456698e-02,-0.039898,-0.588728,-0.103507,0.007464,-4.409571e-01,-3.293650e-01,-3.456698e-02
75%,8.145791,1.0,1.250000,1.307692,2.750000,1.636364,2.839583,5.311782e-01,3.265068e-01,7.406407e-01,7.406407e-01,5.102201e-01,0.540524,0.600480,0.223576,0.411626,2.036267e-01,4.337294e-01,5.102201e-01
max,8.618754,1.0,2.538462,2.750000,4.818182,3.545455,4.344444,2.743110e+00,4.132643e+00,5.222881e+00,5.222881e+00,3.016704e+00,12.421131,4.632471,1.728438,6.602834,5.002195e+00,6.156938e+00,3.016704e+00


In [41]:
g_depovic = smf.ols(formula=f'std_mt2psmfq ~ std_mt1psmfq + c_overt + c_negemot + overtxnegemot', data=girls).fit()
g_depovic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           std_mt2psmfq   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     19.64
Date:                Fri, 05 Apr 2019   Prob (F-statistic):           1.11e-12
Time:                        16:30:11   Log-Likelihood:                -160.53
No. Observations:                 136   AIC:                             331.1
Df Residuals:                     131   BIC:                             345.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.0318      0.070     -0.457      0.649      -0.170       0.106
std_mt1psmfq      0.1363      0.092      1.479      0.141      -0.046       0.319
c_overt           0.0787      0.046      1.728      0.086      -0.011       0.169
c_negemot         0.3291      0.083      3.965      0.000       0.165       0.493
overtxnegemot     0.1029      0.036      2.839      0.005       0.031       0.175
==============================================================================
Omnibus:                       66.589   Durbin-Watson:                   2.330
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.007
Skew:                           1.790   Prob(JB):                     1.28e-57
Kurtosis:                       8.782   Cond. No.                         3.81
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Conclusion...

The t-value and p-value are the same.
Coefficients and standard errors are different.